<a href="https://colab.research.google.com/github/fairml-research/HGR_NN/blob/main/FAIR_HGR_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone "https://github.com/fairml-research/HGR_NN.git"
import sys, os
os.chdir('HGR_NN')

Cloning into 'HGR_NN'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 9), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (22/22), 79.66 KiB | 2.84 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [4]:
from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F

import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from torch.autograd import Variable



### 1) **IMPORT THE DATA**

In [5]:
!wget "https://raw.githubusercontent.com/SoftStackFactory/data-science-fall2019/master/data/census_2015/acs2015_census_tract_data.csv"

--2024-01-04 21:28:30--  https://raw.githubusercontent.com/SoftStackFactory/data-science-fall2019/master/data/census_2015/acs2015_census_tract_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14448669 (14M) [text/plain]
Saving to: ‘acs2015_census_tract_data.csv’

acs2015_census_trac 100%[===================>]  13.78M  --.-KB/s    in 0.1s    

2024-01-04 21:28:31 (130 MB/s) - ‘acs2015_census_tract_data.csv’ saved [14448669/14448669]



In [6]:
import pandas as pd
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..')))

def load_USCENSUS_data(path):
#    column_names = ["age","sex","bmi","children","smoker","region","charges"]
    input_data = pd.read_csv(path, sep=',')
    input_data=input_data.drop(columns=['CensusTract','County'])
    input_data=input_data.dropna()
    input_data['Women']=input_data['Women']/input_data['TotalPop']
    input_data.dtypes
    sensitive_attribs = ['Women']
    Z = (input_data.loc[:, sensitive_attribs])

    y = input_data['ChildPoverty']

    # features; note that the 'target' and sentive attribute columns are dropped
    X = (input_data
         .drop(columns=['Poverty', 'ChildPoverty','Women','Men'])
         .pipe(pd.get_dummies))

    print(f"features X: {X.shape[0]} samples, {X.shape[1]} attributes")
    print(f"targets y: {y.shape[0]} samples")
    print(f"sensitives Z: {Z.shape[0]} samples, {Z.shape[1]} attributes")
    return X, y, Z


# load ICU data set
X, y, Z = load_USCENSUS_data('acs2015_census_tract_data.csv')

# split into train/test set
X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(X, y, Z, test_size=0.2, random_state=7)

# standardize the data
scaler = StandardScaler().fit(X_train)
scale_df = lambda df, scaler: pd.DataFrame(scaler.transform(df), columns=df.columns, index=df.index)
X_train = X_train.pipe(scale_df, scaler)
X_test = X_test.pipe(scale_df, scaler)

meanYtrain=np.mean(y_train)
stdYtrain=np.std(y_train)
y_train=(y_train-meanYtrain)/stdYtrain
y_test =(y_test-meanYtrain)/stdYtrain
s_train=s_train.values.squeeze(1)
s_test=s_test.values.squeeze(1)

features X: 72727 samples, 82 attributes
targets y: 72727 samples
sensitives Z: 72727 samples, 1 attributes


###2) UNFAIR PREDICTOR MODEL

In [22]:
from HGR_NN.Fair_hgr_nn import FAIR_HGR_NN

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
H = 15
H2 = 15




class Predictor(nn.Module):
    def __init__(self):
        super(Predictor, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 1)
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        #x = F.dropout(x, p=0.8)
        x = self.fc2(x)
        x = F.relu(x)
        #x = F.dropout(x, p=0.8)
        x = self.fc3(x)
        x = F.relu(x)
        #x = F.dropout(x, p=0.8)
        x = self.fc4(x)
        return x


class Net_HGR(nn.Module):
    def __init__(self):
        super(Net_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4
class Net2_HGR(nn.Module):
    def __init__(self):
        super(Net2_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4
import torch

device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

lamb_HGR=0 #  so unfair model since no loop over HGR (start_epochHGR >= nbepoch) and LAMBDA = 0
UNFair_NN_S0 = FAIR_HGR_NN(regressor="rmse",mod_h=Predictor, p_device =device,
                            nbepoch =1000 , lr = 0.0005, lambdaHGR=lamb_HGR, nbepochHGR=5, start_epochHGR=1000
                              ,mod_HGR_F=Net_HGR,mod_HGR_G= Net2_HGR, init_HGR=False)

UNFair_NN_S0.fit(X_train, y_train, s_train)



Bar desc (file 1000): 100%|██████████| 1001/1001 [00:09<00:00, 102.97it/s]


array([ 0.6278832 , -0.33456907,  0.1226691 , ...,  0.0171999 ,
        1.1594708 , -0.9362    ], dtype=float32)

In [23]:


H = 15
H2 = 15


class Net_HGR(nn.Module):
    def __init__(self):
        super(Net_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4


class Net2_HGR(nn.Module):
    def __init__(self):
        super(Net2_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4

from HGR_NN.functions import *

y_pred = UNFair_NN_S0.predict(X_train)
y_predt = UNFair_NN_S0.predict(X_test)
y_pred_np=y_pred.cpu().detach().numpy().squeeze(1)
y_predt_np=y_predt.cpu().detach().numpy().squeeze(1)
MSE = round(np.mean((y_train-y_pred_np)**2),4)
MSEt = round(np.mean((y_test-y_predt_np)**2),4)

HGR_NNP = HGR_NN(Net_HGR(),Net2_HGR(),device, display=False)
print("MSE Test", MSEt)
print("HGR NN Test",HGR_NNP(y_predt_np, s_test,1000 ))
print("HGR RDC Test", rdc(y_predt_np, s_test))
print("HGR KDE Test",hgr(y_predt.squeeze(1).to("cpu"), torch.FloatTensor(s_test).to("cpu"),kde).data.numpy())
print("FairQuant Test", FairQuant(s_test,y_test,y_predt_np))

MSE Test 0.2922
HGR NN Test 0.2386583
HGR RDC Test 0.21326508678638006
HGR KDE Test 0.17493503
FairQuant Test 0.07343925915658474



### 3) A QUICK FAIR MODEL VIA HGR NN MINIMIZATION (no need of GPU)
(Quick since there are only 50 epochs of the HGR minimization and no HGR Initialization  (1000 when starting))
--> please note that with only 50 epochs the performance are a little bit less robust and less performant. In section 4) you will train a more robust model with 500 epochs on the HGR (better with GPU! )

In [12]:

from HGR_NN.Fair_hgr_nn import FAIR_HGR_NN


class Predictor(nn.Module):
    def __init__(self):
        super(Predictor, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 1)
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        #x = F.dropout(x, p=0.8)
        x = self.fc2(x)
        x = F.relu(x)
        #x = F.dropout(x, p=0.8)
        x = self.fc3(x)
        x = F.relu(x)
        #x = F.dropout(x, p=0.8)
        x = self.fc4(x)
        return x
H = 15
H2 = 15


class Net_HGR(nn.Module):
    def __init__(self):
        super(Net_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4
class Net2_HGR(nn.Module):
    def __init__(self):
        super(Net2_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4

device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

lamb_HGR=2.5
Fair_HGR_NN_S0 = FAIR_HGR_NN(regressor="rmse",mod_h=Predictor,  p_device =device,
                            nbepoch =1000 , lr = 0.0005, lambdaHGR=lamb_HGR, nbepochHGR=50, start_epochHGR=950
                              ,mod_HGR_F=Net_HGR,mod_HGR_G= Net2_HGR, init_HGR= False)


Fair_HGR_NN_S0.fit(X_train, y_train, s_train)


Bar desc (file 1000): 100%|██████████| 1001/1001 [00:19<00:00, 51.15it/s]


array([ 0.50174505,  0.17450356,  0.03181195, ...,  0.19036525,
        0.73147553, -0.9183152 ], dtype=float32)

In [ ]:

H = 15
H2 = 15


class Net_HGR(nn.Module):
    def __init__(self):
        super(Net_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4


class Net2_HGR(nn.Module):
    def __init__(self):
        super(Net2_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4

from HGR_NN.functions import *
from HGR_NN.functions import HGR_NN

y_pred = Fair_HGR_NN_S0.predict(X_train)
y_predt = Fair_HGR_NN_S0.predict(X_test)

y_pred_np=y_pred.cpu().detach().numpy().squeeze(1)
y_predt_np=y_predt.cpu().detach().numpy().squeeze(1)
MSE = round(np.mean((y_train-y_pred_np)**2),4)
MSEt = round(np.mean((y_test-y_predt_np)**2),4)

HGR_NNP = HGR_NN(Net_HGR(),Net2_HGR(),device, display=False)
print("MSE Test", MSEt)
print("HGR NN Test",HGR_NNP(y_predt_np, s_test,100))
print("HGR RDC Test", rdc(y_predt_np, s_test))
print("HGR KDE Test",hgr(y_predt.squeeze(1).to("cpu"), torch.FloatTensor(s_test).to("cpu"),kde).data.numpy())
print("FairQuant Test", FairQuant(s_test,y_test,y_predt_np))

MSE Test 0.5294
HGR NN Test 0.06170071
HGR RDC Test 0.050341320491113
HGR KDE Test 0.05167916
FairQuant Test 0.0236079566180706



### 4) A MORE ROBUST FAIR MODEL (Better with GPU)
"Init_HGR=True" allows to make a better HGR Initialization  (1000 epochs when it starts mitigating with HGR)
--> please note that there will be now 500 epochs of HGR minimization (instead of 50 as before). Also note that for each epoch 50 epochs of HGR are applied! so it can be long and a GPU can help (with device = 'cuda:0')


In [18]:

from HGR_NN.Fair_hgr_nn import FAIR_HGR_NN


class Predictor(nn.Module):
    def __init__(self):
        super(Predictor, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 1)
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        #x = F.dropout(x, p=0.8)
        x = self.fc2(x)
        x = F.relu(x)
        #x = F.dropout(x, p=0.8)
        x = self.fc3(x)
        x = F.relu(x)
        #x = F.dropout(x, p=0.8)
        x = self.fc4(x)
        return x
H = 15
H2 = 15


class Net_HGR(nn.Module):
    def __init__(self):
        super(Net_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4
class Net2_HGR(nn.Module):
    def __init__(self):
        super(Net2_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4

device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

lamb_HGR=3
Fair_HGR_NN_S1 = FAIR_HGR_NN(regressor="rmse",mod_h=Predictor,  p_device =device,
                            nbepoch =1500 , lr = 0.0005, lambdaHGR=lamb_HGR, nbepochHGR=50, start_epochHGR=1000
                              ,mod_HGR_F=Net_HGR,mod_HGR_G= Net2_HGR,init_HGR=True)


Fair_HGR_NN_S1.fit(X_train, y_train, s_train)


Bar desc (file 1500): 100%|██████████| 1501/1501 [02:02<00:00, 12.23it/s]


array([ 0.7393369 , -0.02141701, -0.13145773, ...,  0.23513974,
        0.21242495, -0.9526018 ], dtype=float32)

In [19]:

H = 15
H2 = 15


class Net_HGR(nn.Module):
    def __init__(self):
        super(Net_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4


class Net2_HGR(nn.Module):
    def __init__(self):
        super(Net2_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4

from HGR_NN.functions import *
from HGR_NN.functions import HGR_NN

y_pred = Fair_HGR_NN_S1.predict(X_train)
y_predt = Fair_HGR_NN_S1.predict(X_test)

y_pred_np=y_pred.cpu().detach().numpy().squeeze(1)
y_predt_np=y_predt.cpu().detach().numpy().squeeze(1)
MSE = round(np.mean((y_train-y_pred_np)**2),4)
MSEt = round(np.mean((y_test-y_predt_np)**2),4)

HGR_NNP = HGR_NN(Net_HGR(),Net2_HGR(),device, display=False)
print("MSE Test", MSEt)
print("HGR NN Test",HGR_NNP(y_predt_np, s_test,100))
print("HGR RDC Test", rdc(y_predt_np, s_test))
print("HGR KDE Test",hgr(y_predt.squeeze(1).to("cpu"), torch.FloatTensor(s_test).to("cpu"),kde).data.numpy())
print("FairQuant Test", FairQuant(s_test,y_test,y_predt_np))

MSE Test 0.5213
HGR NN Test 0.05264287
HGR RDC Test 0.04489993367342854
HGR KDE Test 0.047230523
FairQuant Test 0.01774695102125406
